In [3]:
import os

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Text-Summarisation-using-NLP'

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [8]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path= config.data_path,
            tokenizer_name= config.tokenizer_name
        )

        return data_transformation_config


In [10]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [11]:
import transformers
import datasets

[2024-03-31 23:55:22,134: INFO:config:PyTorch version 2.2.2 available.]


In [12]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import  load_dataset, load_from_disk

In [15]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, data_in_batch):
        input_encoding = self.tokenizer(data_in_batch["dialogue"], max_length =1024, truncation=True)
       
        target_encoding = self.tokenizer(data_in_batch["summary"], max_length =128, truncation=True)

        return{
            "input_ids":input_encoding['input_ids'],
            "attention_mask": input_encoding['attention_mask'],
            "labels": target_encoding['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        data_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched= True)
        data_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))




In [16]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config= data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e

[2024-03-31 23:57:54,708: INFO:common:config\config.yaml loaded succcessfully]
[2024-03-31 23:57:54,713: INFO:common:params.yaml loaded succcessfully]
[2024-03-31 23:57:54,716: INFO:common:created directory at: artifacts]
[2024-03-31 23:57:54,718: INFO:common:created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]